## The purpose of this exercise is to download an article from a Washington Post URL and use natural language processing (NLP) to summarize it in three sentences.

In [10]:
from nltk.tokenize import sent_tokenize, word_tokenize # Split text into sentences, and sentences into words.
from nltk.corpus import stopwords # Used to filter out common words ("and", "but", "I", "the", etc.)
from string import punctuation # Used to filter out punctuation.
from collections import defaultdict # Dictionary that creates entries for new keys instead of throwing a KeyError.
from heapq import nlargest # Returns the 'n' largest items from a list based on a method.

# Uses ntlk to summarize a set of text based on word frequency.
class FrequencySummarizer:
    
    def __init__(self, min_cut=0.1, max_cut=0.9):
        # Words with a frequency term outside of the range between min_cut and max_cut will be ignored.
        self._min_cut = min_cut
        self._max_cut = max_cut
        # Common words and symbols to be ignored for summarization.
        self._stopwords = set(stopwords.words('english')+list(punctuation))
    
    # Accepts a list of sentences. 
    # Returns a dictionary of word:frequency pairs.
    def _compute_frequencies(self, word_sent):
        # Create a defaultdict (see import at top) with 0 as the default value.
        freq = defaultdict(int)
        # Loop through each word of each sentence to count their frequencies.
        for sentence in word_sent:
            for word in sentence:
                if word not in self._stopwords:
                    # Note: Using the regular dictionary would require checking if the key exists first.
                    # because freq is a defaultdict, it will create an entry the first time a word is added.
                    freq[word] += 1
        # Normalize the frequencies so they fall between 0 and 1 by dividing the frequency of every word by
        # the largest frequency in the dictionary.
        max_n = float(max(freq.values()))
        for word in list(freq.keys()):
            freq[word] = freq[word]/max_n 
            # Frequencies outside of max_cut and min_cut are also filtered out in this step.
            if freq[word] >= self._max_cut or freq[word] <= self._min_cut:
                del freq[word]
        # Return the dictionary of frequencies.
        return freq
    
    # Accepts text and the number of sentences the summary should contain.
    # Returns a summary of the text.
    def summarize(self, text, n):
        # Split the text into sentences (see import at top)
        sentences = sent_tokenize(text)
        # Sanity check to make sure the summary is less than the length of the article.
        assert n <= len(sentences)
        # Compile each word of each sentence into a single dictionary.
        # For consistency, they're all converted to lowercase.
        word_sent = [word_tokenize(sentence.lower()) for sentence in sentences]
        # Compute the word frequencies (see above method) and store in member variable _freq.
        self._freq = self._compute_frequencies(word_sent)
        # Create a defaultdict to rank sentences by frequency.
        rankings = defaultdict(int)
        # First, enumerate each of the stentences for easier ranking.
        for i,sentence in enumerate(word_sent):
            # For each word in a sentence...
            for word in sentence:
                # ...if that word wasn't a stopword...
                if word in self._freq:
                    # ...add its frequency to the ranking.
                    rankings[i] += self._freq[word]
        # Fetch the indexes of the largest n sentences.
        # nlargest (see import at top) is given ranking.get() to know which values to rank.
        sentences_index = nlargest(n, rankings, key=rankings.get)
        # Return a list containing the top sentences.
        return [sentences[j] for j in sentences_index]

In [20]:
import urllib.request # Downloads URLs.
from bs4 import BeautifulSoup # Parses html pages in an easy-to-use way.

# Accepts a URL for a Washington Post article.
# Returns a title,body pair containing only text from the article.
def get_text_from_wapo_url(url):
    # Download the URL. Assumed to use utf-8 encoding.
    page = urllib.request.urlopen(url).read().decode('utf8')
    # Initialize a BeautifulSoup (see imports above) with url's text.
    soup = BeautifulSoup(page, 'html.parser')
    # WaPo wraps all of their articles in <article> tags, which is used
    # to search for them here via BeautifulSoup.
    text = ' '.join(map(lambda p: p.text, soup.find_all('article')))
#     print(text)
    # Since other stuff is usually included between the <article> tags,
    # another pass is necessary to filter to just article text. This is
    # done by searching that text for <p> (paragraph) tags.
#     soup2 = BeautifulSoup(text, 'html.parser')
#     text = ' '.join(map(lambda p: p.text, soup2.find_all('p')))
#     print(text)
    #Return the title of the article and its text.
    return soup.title.text, text

In [23]:
someUrl = input()
urlText = get_text_from_wapo_url(someUrl)
fs = FrequencySummarizer()
summary = fs.summarize(urlText[1], 3)
print(summary)

https://www.washingtonpost.com/news/post-nation/wp/2016/11/26/clinton-campaign-will-participate-in-wisconsin-recount-with-an-eye-on-outside-interference-lawyer-says/?hpid=hp_hp-top-table-main_pn-recount-1234pm%3Ahomepage%2Fstory
['In a Medium post, Clinton campaign lawyer Marc Elias said that the campaign had received “hundreds of messages, emails, and calls urging us to do something, anything, to investigate claims that the election results were hacked and altered in a way to disadvantage Secretary Clinton,” especially in Michigan, Wisconsin and Pennsylvania, where the “combined margin of victory for Donald Trump was merely 107,000 votes.” Elias said the campaign had “not uncovered any actionable evidence of hacking or outside attempts to alter the voting technology.” But because of the margin of victory — and because of the degree of apparent foreign interference during the campaign — Elias said that Clinton officials had “quietly taken a number of steps in the last two weeks to rule